># [Build an Agent](https://python.langchain.com/docs/tutorials/agents/)
> ---
> *Build agent that can interact with search engine*

Ask this agent questions, watch it call the search tool, and have conversations with it

* [Agents](https://python.langchain.com/docs/concepts/agents/) are systems using LLMs as**reasoning engines to determine:
  *  `action(s)`
  *  `inputs` needed for this

* After executing actions results fed back into LLM:
  *  Det `next_action(s)` and/or
  *  Det `finished`?

Often acheived via [tool-calling](https://python.langchain.com/docs/concepts/tool_calling/)

## End-to-end agent

* Fully functional agent using LLM to decide which tools to use
* Equipped with a **generic search tool**. 
* Has conversational memory → **multi-turn chatbot**

In [ ]:
import os
from dotenv import load_dotenv

# Load secrets from file
with open('secrets.txt') as f:
    for line in f:
        if '=' in line:
            key, value = line.strip().split('=', 1)
            os.environ[key] = value

# Initialize LangChain
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4")

In [16]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(
        content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hello Bob! It's nice to meet you. San Francisco is a wonderful city with so much to see and do. What are some of your favorite things about living there? I'd be happy to provide some recommendations for attractions, restaurants, or activities if you're looking for new things to check out in the area.", additional_kwargs={}, response_metadata={'id': 'msg_01DTPD9YzGFNQU7gWX9AVWYR', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 271, 'output_tokens': 67}}, id='run-5f6ccb76-811a-4244-b862-c751fd67677e-0', usage_metadata={'input_tokens': 271, 'output_tokens': 67, 'total_tokens': 338, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})]}}
----
{'agent': {'messages': [AIMessage(content=[{'text': 'Okay, let me use the search tool to look up the current weather forecast for San Francisco:', 'type': 'text'}, {'id'

## Define Tools

main tool of choice will be [Tavily](https://python.langchain.com/docs/integrations/tools/tavily_search/) - a search engine. We have a built-in tool in LangChain to easily use Tavily search engine as tool.

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

In [17]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

## Using Language Models to Call Tools


You can call the language model by passing in a **list of messages**. By default, the **response is a content string**.

In [19]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

### Use `.bind_tools` to give LLM knowledge of these tools

In [20]:
model_with_tools = model.bind_tools(tools)

##### Not using tools (no need from query)

In [21]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


##### With tool use

In [ ]:
response = model_with_tools.invoke(
    [HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_XASeUbhgncRfNtN1xLmu0Kab', 'type': 'tool_call'}]


* no text content, but there is a tool call! 

* isn't calling that tool yet - it's just telling us to

* need to agent

## Creating `agent`

```
LLM + tools = agent
````

  * using a high level interface to construct `agent`,* LangGraph has high-level interface backed by low-level → highly controllable API → can modify gent logic


initialize the agent with  `LLM` & `tools`

> `Note:`  we are passing  `model` not `model_with_tools` ; `create_react_agent` will call `.bind_tools` under the hood

In [23]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

## Run `agent`

Running stateless queries (won't remember previous interactions)

 agent will return  final state at the end of the interaction (which includes any inputs, we will see later on how to get only the outputs).


In [24]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='94e3641e-8b35-4b9c-8215-7a2e6a15fec6'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 83, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0da627c9-2a8b-499b-874c-74cc866c83e2-0', usage_metadata={'input_tokens': 83, 'output_tokens': 11, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

 now try it out on an example where it should be invoking the tool



In [25]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='c220ce71-3bf7-4dd8-99d7-f73a217925d8'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ctUd7YAPgfO5WfOwF0QsygQf', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 88, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3c509336-d5dc-421f-9833-d925f9119b1d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_ctUd7YAPgfO5WfOwF0QsygQf